In [26]:
%cd /Users/annama/Documents/Master/SS 25/Data Analytics in Applications/Code/daia-eon/daia-eon-1/notebooks


/Users/annama/Documents/Master/SS 25/Data Analytics in Applications/Code/daia-eon/daia-eon-1/notebooks


import re, json
import pandas as pd
from pathlib import Path

# Finegrained placeholder map
PLACEHOLDERS = {
    "VORNAME": {"VORNAME"},
    "NACHNAME": {"NACHNAME"},
    "FIRMA": {"FIRMA"},
    "TELEFONNUMMER": {"TELEFONNUMMER"},
    "EMAIL": {"EMAIL"},
    "FAX": {"FAX"},
    "STRASSE": {"STRASSE"},
    "HAUSNUMMER": {"HAUSNUMMER"},
    "POSTLEITZAHL": {"POSTLEITZAHL", "PLZ","ZIP"},
    "WOHNORT": {"WOHNORT", "ORT","CITY"},
    "ZÄHLERNUMMER": {"ZÄHLERNUMMER", "METER_ID"},
    "ZÄHLERSTAND": {"ZÄHLERSTAND", "METER_READING"},
    "VERTRAGSNUMMER": {"VERTRAGSNUMMER", "ANGEBOTSNUMMER", "KUNDENNUMMER"},
    "ZAHLUNG": {"BETRAG","ZAHLUNG","AMOUNT"},
    "BANK": {"BANK"},
    "IBAN": {"IBAN"},
    "BIC": {"BIC"},
    "DATUM": {"DATUM","DATE"},
    "TITEL": {"TITEL"},
    "GESENDET_MIT": {"GESENDET_MIT"},
    "LINK": {"LINK"},
}


def map_col(col: str) -> str|None:
    up = col.upper()
    for tag, keys in PLACEHOLDERS.items():
        if any(k in up for k in keys):
            return f"<<{tag}>>"
    return None

def extract_repls(row: pd.Series):
    repl = []
    for col, val in row.items():
        if pd.isna(val): continue
        ph = map_col(col)
        if not ph:    continue
        literal = str(val).strip()
        if literal:
            repl.append((re.escape(literal), ph))
    return sorted(repl, key=lambda x: len(x[0]), reverse=True)


In [30]:
META       = Path("data/Daia_Manual_Labelling_granular.xlsx")
RAW_DIR    = Path("data/golden_dataset_original")
JSON_OUT   = Path("data/golden_dataset_with_spans.json")

meta    = pd.read_excel(META, dtype=str)
records = []
tag_re  = re.compile(r"<<([^>]+)>>")  # capture inside the chevrons

for _, row in meta.iterrows():
    fn   = row["TextFile"]
    path = RAW_DIR/fn
    if not path.exists(): 
        print("⚠️ Missing", fn)
        continue

    text   = path.read_text(encoding="utf-8", errors="ignore")
    labels = []

    # for each literal→<<TAG>> pairing, find its spans in the original text
    for pattern, placeholder in extract_repls(row):
        tag = tag_re.match(placeholder).group(1)  # e.g. "VORNAME"
        for m in re.finditer(pattern, text, flags=re.IGNORECASE):
            labels.append({
                "start": m.start(),
                "end":   m.end(),
                "label": tag
            })

    records.append({
        "file":   fn,
        "text":   text,
        "labels": labels
    })

JSON_OUT.write_text(json.dumps(records, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"✓ Wrote {len(records)} records to {JSON_OUT}")

✓ Wrote 160 records to data/golden_dataset_with_spans.json


In [33]:
import re
import json
import pandas as pd
from pathlib import Path

# ── 0) Assume you already have these from your TXT script ─────────────
PLACEHOLDERS = {
    "TITEL":         ["TITEL"],
    "VORNAME":       ["VORNAME"],
    "NACHNAME":      ["NACHNAME"],
    "FIRMA":         ["FIRMA"],
    "TELEFONNUMMER": ["TELEFONNUMMER"],
    "EMAIL":         ["EMAIL"],
    "FAX":           ["FAX"],
    "STRASSE":       ["STRASSE"],
    "HAUSNUMMER":    ["HAUSNUMMER"],
    "POSTLEITZAHL":  ["POSTLEITZAHL","PLZ","ZIP"],
    "WOHNORT":       ["WOHNORT","ORT","CITY"],
    "ZÄHLERNUMMER":  ["ZÄHLERNUMMER","METER_ID"],
    "ZÄHLERSTAND":   ["ZÄHLERSTAND","METER_READING"],
    "VERTRAGSNUMMER":["VERTRAGSNUMMER","ANGEBOTSNUMMER","KUNDENNUMMER", "RECHNUNGSNUMMER"],
    "ZAHLUNG":       ["BETRAG","ZAHLUNG","AMOUNT"],
    "BANK":          ["BANK"],
    "IBAN":          ["IBAN"],
    "BIC":           ["BIC"],
    "DATUM":         ["DATUM","DATE"],
    "GESENDET_MIT":  ["GESENDET_MIT"],
    "LINK":          ["LINK"],
}

def map_col(col: str) -> str|None:
    up = col.upper()
    for tag, keys in PLACEHOLDERS.items():
        if any(k in up for k in keys):
            return f"<<{tag}>>"
    return None

def extract_repls(row: pd.Series):
    repl = []
    for col, val in row.items():
        if pd.isna(val): continue
        ph = map_col(col)
        if not ph:    continue
        lit = str(val).strip()
        if lit:
            repl.append((re.escape(lit), ph))
    return sorted(repl, key=lambda x: len(x[0]), reverse=True)

# ── 1) Paths & load metadata ──────────────────────────────────────────
META      = Path("data/Daia_Manual_Labelling_granular.xlsx")
RAW_DIR   = Path("data/golden_dataset_original")
JSON_OUT  = Path("data/original_with_spans.json")

df = pd.read_excel(META, dtype=str)
tag_re = re.compile(r"<<([^>]+)>>")

output = []

# ── 2) For each email, apply the same patterns to the original text ───
for _, row in df.iterrows():
    fname    = row["TextFile"]
    orig_txt = (RAW_DIR/fname).read_text(encoding="utf-8", errors="ignore")

    labels = []
    for pattern, placeholder in extract_repls(row):
        tag = tag_re.match(placeholder).group(1)  # e.g. "VORNAME"
        # find all occurrences in the original
        for m in re.finditer(pattern, orig_txt, flags=re.IGNORECASE):
            labels.append({
                "start": m.start(),
                "end":   m.end(),
                "label": tag
            })

    output.append({
        "file":   fname,
        "text":   orig_txt,
        "labels": labels
    })

# ── 3) Write out ───────────────────────────────────────────────────────
JSON_OUT.write_text(json.dumps(output, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"✓ Wrote {len(output)} records to {JSON_OUT}")


✓ Wrote 160 records to data/original_with_spans.json
